<a href="https://colab.research.google.com/github/tomohitom/Tennis_Game_Analysis/blob/master/Openpose.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone "https://github.com/YutaroOgawa/pytorch_advanced.git"

Cloning into 'pytorch_advanced'...
remote: Enumerating objects: 529, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 529 (delta 23), reused 32 (delta 13), pack-reused 479
Receiving objects: 100% (529/529), 17.57 MiB | 29.94 MiB/s, done.
Resolving deltas: 100% (282/282), done.


In [ ]:
!pip install ffmpeg-python

In [6]:
!python /content/pytorch_advanced/4_pose_estimation/make_folders.py

COCO.jsonとmask.tar.gzを「data」ディレクトリにアップロード

In [8]:
!python /content/pytorch_advanced/4_pose_estimation/data_downloads.py